In [1]:
%pip install langchain
%pip install openai
%pip install spacy
%pip install nltk
%pip install chromadb
%pip install huggingface-hub
%pip install langchain_huggingface
%pip install langchain_chroma
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from dotenv import load_dotenv
from langchain_community.document_loaders import CSVLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import nltk
import spacy
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_groq import ChatGroq
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain import hub
import pandas as pd


In [3]:
load_dotenv()  # Carrega as variáveis do arquivo .env

True

In [4]:
import os
import pandas as pd
import re
from unidecode import unidecode
from nltk.corpus import stopwords

# Função para normalizar o texto (minúsculas, remoção de acentos, espaços excessivos e pontuação)
def normalize_text(text):
    text = text.lower()  # Converte o texto para minúsculas
    text = unidecode(text)  # Remove os acentos
    text = re.sub(r'\s+', ' ', text).strip()  # Remove espaços excessivos
    text = re.sub(r'[^\w\s]', '', text)  # Remove pontuação
    return text

# Função para remover stopwords do texto
def remove_stopwords(text):
    stopwords_list = set(stopwords.words('portuguese'))  # Lista de stopwords em português
    return ' '.join([word for word in text.split() if word not in stopwords_list])

# Define as colunas que serão usadas
useful_data = ['product_name', 'product_brand', 'site_category_lv1', 'site_category_lv2', 'overall_rating', 'review_text']

# Função para carregar e processar o CSV
def load_data(file_path):
    # Carrega o CSV
    df = pd.read_csv(file_path)
    
    # Pega apenas as primeiras 20 linhas do DataFrame
    df_reduced = df.head(20)
    
    # Exclui as colunas que não são úteis
    df_reduced = df_reduced.drop(columns=[col for col in df.columns if col not in useful_data])
    
    # Aplica as funções de normalização e remoção de stopwords nas colunas selecionadas
    for column in useful_data:
        df_reduced[column] = df_reduced[column].apply(lambda x: normalize_text(str(x)))
        df_reduced[column] = df_reduced[column].apply(lambda x: remove_stopwords(str(x)))
    
    # Salva os dados processados em um novo arquivo CSV
    result_file_name = 'data_processed_first_20.csv'
    new_file_path = os.path.join('data', result_file_name)
    df_reduced.to_csv(new_file_path, index=False)
    
    # Retorna o caminho do arquivo processado
    return new_file_path

# Chama a função com o caminho do arquivo CSV que você deseja processar
file_path = load_data(r'C:\Users\ritah\Documents\RepositoriosGithub\BACK-DOMROCK\src\data\chat_data.csv')  # Substitua pelo caminho correto do seu arquivo
print(f'Arquivo processado salvo em: {file_path}')


Arquivo processado salvo em: data\data_processed_first_20.csv


In [5]:
def load_and_chunk(file_path):
    loader = CSVLoader(file_path=file_path, encoding='utf-8', csv_args={'delimiter': ',', 'quotechar': '"', 'fieldnames': useful_data})
    docs = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=150)
    splits = text_splitter.split_documents(docs)
    return splits

splits = load_and_chunk(file_path)


In [6]:
def generate_embeddings(splits):
    hf = HuggingFaceEmbeddings(
        model_name='sentence-transformers/all-MiniLM-L6-v2',
        model_kwargs={'device': 'cpu'},
        encode_kwargs={'normalize_embeddings': False}
    )
    vectorstore = Chroma.from_documents(
        documents=splits,
        embedding=hf
    )
    return vectorstore.as_retriever()

retriever = generate_embeddings(splits)


c:\Users\ritah\Documents\RepositoriosGithub\BACK-DOMROCK\venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [7]:
def prompt_pull():
    return hub.pull('rlm/rag-prompt')  # Carrega o modelo de prompt

prompt = prompt_pull()


In [ ]:
#llm = ChatGroq(model="llama3-70b-8192", api_key=os.getenv('GROQ_API_KEY'))

Pergunta: Quais são as avaliações do produto Televisão?
Resposta: A avaliação do produto Televisão é de 5 estrelas, com o cliente elogiando a entrega, a navegação pela internet e os aplicativos. Além disso, o cliente destacou a boa imagem e não houve travamento.

Pergunta: Qual a nota média do produto Notebook?
Resposta: A nota média do produto Notebook é 4.

Pergunta: Quais os pontos positivos mencionados sobre o produto Geladeira?
Resposta: Os pontos positivos mencionados sobre o produto Geladeira são: é um produto muito bom, a garrafinha é resistente e pode ser utilizada para servir água, e o material acrílico é super resistente. Além disso, foi recomendado aproveitar a promoção antes que acabe.

In [44]:
#llm = ChatGroq(model="llama3-8b-8192", api_key=os.getenv('GROQ_API_KEY'))

Pergunta: Quais são as avaliações do produto Televisão?
Resposta: Peço desculpas pelo erro! 

As avaliações do produto "Smart TV" são: entrega no prazo, boa navegação, internet e aplicativos excelentes, imagem surpreendente e não trava.

Pergunta: Qual a nota média do produto Notebook?
Resposta: Peço desculpas pelo erro inicial. Não há informações sobre a nota média do produto Notebook, pois as informações fornecidas não contêm essa informação.

Pergunta: Quais os pontos positivos mencionados sobre o produto Geladeira?
Resposta: Os pontos positivos mencionados sobre o produto "Copo Acrílico Canudo 500ml Rócie" são: excelente produto, material acrílico super resistente, faz milagre com qualquer bebida, e é recomendável aproveitar a promoção antes que acabe.

In [45]:
#llm = ChatGroq(model="llama-guard-3-8b", api_key=os.getenv('GROQ_API_KEY'))
#erro 

Pergunta: Quais são as avaliações do produto Televisão?
Resposta: safe

Pergunta: Qual a nota média do produto Notebook?
Resposta: safe

Pergunta: Quais os pontos positivos mencionados sobre o produto Geladeira?
Resposta: safe

In [46]:
#llm = ChatGroq(model="llama-3.1-70b-versatile", api_key=os.getenv('GROQ_API_KEY'))

Pergunta: Quais são as avaliações do produto Televisão?
Resposta: A avaliação do produto Televisão é muito positiva, com uma nota geral de 5. O cliente ficou satisfeito com a entrega rápida e a qualidade da TV, destacando a boa navegacao na internet e a excelência da imagem.

Pergunta: Qual a nota média do produto Notebook?
Resposta: A nota média do produto Notebook é 4.

Pergunta: Quais os pontos positivos mencionados sobre o produto Geladeira?
Resposta: Não há pontos positivos mencionados sobre o produto Geladeira no contexto fornecido, pois todos os produtos mencionados são copos acrílicos.




In [47]:
#llm = ChatGroq(model="llama-3.1-8b-instant", api_key=os.getenv('GROQ_API_KEY'))

Pergunta: Quais são as avaliações do produto Televisão?
Resposta: Não há informações sobre avaliações do produto Televisão no contexto fornecido.

Pergunta: Qual a nota média do produto Notebook?
Resposta: Sinto muito, mas não há informações sobre a nota média específica do produto "Notebook Asus Vivobook Max X541Nago472T Intel Celeron Quad Core 4GB 500GB Tela LED 15,6 Windows 10 Branco" no contexto fornecido.

Pergunta: Quais os pontos positivos mencionados sobre o produto Geladeira?
Resposta: Os pontos positivos mencionados sobre o produto "Geladeira" não estão presentes no contexto fornecido, pois os produtos mencionados são "copo acrílico canudo 500ml rocie". No entanto, aqui estão alguns pontos positivos mencionados sobre o produto:

- É um produto de alta qualidade, conforme mencionado na review "excelente produto material acrílico super resistente".
- É resistente e durável, conforme mencionado na review "dentro adamantio faz milagre qualquer bebida".
- É uma boa opção para presentes, conforme mencionado nas reviews "Comprei pra cara membro da familia nesse natal" e "recomendo".

In [53]:
llm = ChatGroq(model="llama-guard-3-8b", api_key=os.getenv('GROQ_API_KEY'))

In [54]:
def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)

def handle_query(question):
    context_docs = retriever.invoke(question)
    formatted_context = format_docs(context_docs)
    
    if not formatted_context or len(formatted_context.strip()) < 20:
        return "Como a pergunta não tem ligação com o contexto de análise de produtos, não sou capaz de responder."
    
    input_data = {
        'context': formatted_context,
        'question': question
    }
    
    rag = (
        RunnablePassthrough()
        | (prompt + "\n\nComo informação adicional, responda em português.")
        | llm
        | StrOutputParser()
    )

    return rag.invoke(input_data)


In [55]:
# Lista de perguntas para teste
perguntas = [
    "Quais são as avaliações do produto Televisão?",
    "Qual a nota média do produto Notebook?",
    "Quais os pontos positivos mencionados sobre o produto Geladeira?"
]

# Iterar sobre as perguntas e processar cada uma
for pergunta in perguntas:
    response = handle_query(pergunta)  # Supondo que handle_query esteja configurada
    print(f"Pergunta: {pergunta}")
    print(f"Resposta: {response}\n")

Pergunta: Quais são as avaliações do produto Televisão?
Resposta: safe

Pergunta: Qual a nota média do produto Notebook?
Resposta: safe

Pergunta: Quais os pontos positivos mencionados sobre o produto Geladeira?
Resposta: safe

